In [ ]:
import pyperclip
import pyautogui
import time
import sys
import io,os
import browser_cookie3
import requests
import numpy as np
import pandas as pd
import webvtt
import pyktok as pyk
import random
from PIL import Image

# TODO: remove absoulte paths
sys.path.append("C:/Users/swade/reclaim-tiktok")
sys.path.append("C:/Users/swade/transcriber")

We strongly recommend you run 'specify_browser' first, which will allow you to run pyktok's functions without using the browser_name parameter every time. 'specify_browser' takes as its sole argument a string representing a browser installed on your system, e.g. "chrome," "firefox," "edge," etc.


In [2]:
pyk.specify_browser("firefox")

In [3]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import AzureOpenAIEmbeddings

from reclaim_tiktok.classifier.classifier import Classifier

In [6]:
from transcriber.tiktok_video_details import  TiktokVideoDetails

### API keys

# removed for github -> TODO: use .env file 

In [4]:
# os.environ["OPENAI_API_KEY"]= 
# os.environ["NEWSAPI_API_KEY"]= 
# os.environ["AZURE_OPENAI_ENDPOINT"]=
# os.environ["AZURE_OPENAI_API_KEY"]= 

In [5]:
# os.environ["AZURE_SPEECH_KEY"]= 
# os.environ["AZURE_SPEECH_REGION"]= 
# os.environ["AZURE_SUBSCRIPTION_ID"]= 
# os.environ["AZURE_SPEECH_ENDPOINT"]= 
# os.environ["RECLAIM_TIKTOK_ACCOUNT_ID"]= 

# os.environ["RESOURCE_GROUP"]= 
# os.environ["AZURE_SUBSCRIPTION_KEY"]= 
# os.environ["OBJECT_ID"]=
# os.environ["AZURE_VIDEO_ACCESS_TOKEN"]= ""

In [8]:
# Move mouse to specific coordinates and click
def click(x):
    pyautogui.moveTo(x[0], x[1], duration=0.5)  # Move to the specified coordinates
    pyautogui.click()  # Perform a mouse click
    
    
def get_clipboard():
    content= pyperclip.paste()
    return content


# Function to perform a random click within a given range
def random_click(cords):
    x = random.randint(cords["x"][0], cords["x"][1])
    y = random.randint(cords["y"][0], cords["y"][1])
    pyautogui.moveTo(x, y, duration=0.5)
    pyautogui.click()

def move_to_location(x, y):
    pyautogui.moveTo(x, y, duration=0.5)

In [7]:
play_cords= {"x":[1280, 1550],"y":[450, 630]}

### CLASSIFIER

In [9]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-small-eastus",
    openai_api_version="2024-02-01",
)

right_classifier = Classifier(classifier_path="C:/Users/swade/reclaim-tiktok/data/classifier/")

### FUNCTION TO DETECT BUTTONS

In [11]:
copy_img= 'C:/Users/swade/Downloads/copy_link_mobile_app.png'
share_img= 'C:/Users/swade/Downloads/share_img.png'

In [14]:
def detect_button_position_updated(button_image_path, button_type):
    retry_count = 0
    max_retries = 3
    
    while retry_count < max_retries:
        try:
            # Locate the button on the screen
            if button_type == "share":
                time.sleep(2)
                button_position = pyautogui.locateOnScreen(button_image_path, confidence=0.8, region=(1590, 470, 70, 400))
            else:
                time.sleep(2)
                button_position = pyautogui.locateOnScreen(button_image_path, confidence=0.85)
            
            # If button is found, return its position
            button_x, button_y = pyautogui.center(button_position)
            return [button_x, button_y]
        
        except pyautogui.ImageNotFoundException:
            retry_count += 1
            print("Button image not found. Retrying... ({} Attempt {}/{})".format(button_type, retry_count, max_retries))
            time.sleep(1)  # Wait for 1 second before retrying
    
    # If button is not found after all retries, print a message and return None
    print("Button not found after {} retries.".format(max_retries))
    return None

In [14]:
file_name= "OneDrive/update.csv"

In [34]:
count= 0
api_errors=0
start_time = time.time()
clipboard_content= "empty"
while count< 500:
    
    random_click(play_cords) #pause
    
    # find and click share
    share_cords= detect_button_position_updated(share_img,"share")
    time.sleep(1)
    
    if share_cords is None:
        pyautogui.scroll(-10)
        continue
        
    click(share_cords)

    # find and click copy
    copy_cords= detect_button_position_updated(copy_img,"copy")
    
    if copy_cords is None:
        random_click(play_cords) #tap to go back
        pyautogui.scroll(-10)
        continue
        
    click(copy_cords)
    time.sleep(2)
    
    # get clipboard content
    clipboard_content = get_clipboard()
    
    if clipboard_content == "empty":
        
        pyautogui.scroll(-10)
        print("none link")
        continue
           
    # API call to fetch video metadata
    try:
        video_info= TiktokVideoDetails(clipboard_content)
        time.sleep(1)
        ## transcript
        transcripts= video_info.get_transcriptions()
        time.sleep(1)
        de_transcript= transcripts["deu-DE"]
        
    except Exception as e:
        
        print("error in fetching video data")
        api_errors+= 1
        pyautogui.scroll(-10)
        continue
    
    # suggested words
    
    suggested_words= video_info.suggested_words
    
    suggested_words_string= ' '.join(suggested_words)
    
    # final concatenated text
    text= str(video_info.description) + " " +  str(suggested_words_string) +" " +  str(de_transcript)
    
    # classifier 
    classifier_decision = right_classifier.right_wing_classifier(text, embeddings_client)
 

    
    if classifier_decision == "right":
        
        print(f"Decision of the classifier is: {classifier_decision}. Watching the video")
              
        random_click(play_cords)

        video_info.save_data_to_csv_file(file_name)
        
        time.sleep(min(video_info.duration,60))

        move_to_location(1400, 500)
        pyautogui.scroll(-10)
        
        count += 1
        
    elif classifier_decision == "pluralistic":
        
        print(f"Decision of the classifier is: {classifier_decision}. Skipping the video")
        
        #random_click(play_cords)
        time.sleep(1)
        move_to_location(1400, 500)
        pyautogui.scroll(-10)
        
    
    
    clipboard_content = pyperclip.copy("empty")
    
end_time = time.time()

elapsed_time = end_time - start_time

elapsed_time_minutes = elapsed_time / 60

print("Time taken for the while loop:", elapsed_time_minutes, "minutes")



Decision of the classifier is: right. Watching the video
Decision of the classifier is: right. Watching the video
Decision of the classifier is: pluralistic. Skipping the video
Decision of the classifier is: right. Watching the video
Time taken for the while loop: 3.8561462998390197 minutes
